# Collection of GOES-16 Satellite Images

## Justin Richling
## 09/20/18


# GOES-16 Channels

Plot the 3 main images: 

* Visible - Red (Band 2) 

* Mid-Level Water Vapor (Band 9)

* Clean Infrared (Band 13)

In [ ]:
%matplotlib inline

Imports



In [ ]:
# Random Library Imports
import subprocess,os,glob,tempfile,re,imageio,webbrowser,io,sys,types,urllib,urllib2,\
time,cStringIO

# Importing Datetime Libraries
from datetime import datetime

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Proj 

# Numerical and Scientific Libraries
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

# Accessing Data from External Databases via XLM Catalog
from siphon.ncss import NCSS
from siphon.catalog import TDSCatalog

# MetPy Libraries
import metpy
import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import ctables
from metpy.plots import add_metpy_logo

# NetCDF Libraries
from netCDF4 import Dataset
from netCDF4 import num2date

# More Image Manipulation Options
import PIL
from PIL import Image as PILImage

# Ipyhton Options
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML, display, Image

# Matplotlib Plotting Libraries
#import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm, Normalize
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# Pull the current time
now = datetime.now() 

# Set a path to save the plots with string format for the date to set the month and day 
im_save_path ="/path/to/where/you/want/maps/"+'{0:%m_%d}'.format(now)+"/"
print im_save_path

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

# Create new folders for each channel inside the main folder - for animated gifs
VISPATH = im_save_path+"GOES-VIS/"
WVPATH = im_save_path+"GOES-WV/"
IRPATH = im_save_path+"GOES-IR/"
if not os.path.isdir(VISPATH):
    os.makedirs(VISPATH)

if not os.path.isdir(WVPATH):
    os.makedirs(WVPATH)
    
if not os.path.isdir(IRPATH):
    os.makedirs(IRPATH)

## GOES-16 Data
Channel 13 IR Custom Colorbar to match Typical Color Maps

In [ ]:
# Recreation of University of Wisconsin? IR color bar, thanks to Unidata for making this possible

red = []
green = []
blue = []
valmin=-110.
valmax=55.
red.append( [0.0, 0.0, 0.0] ) 
green.append( [0.0, 0.0, 0.0] )
blue.append( [0.0, 0.0, 0.0] )
red.append( [ (-50.0-valmin)/(valmax-valmin), 0.0, 0.0 ] )
green.append( [ (-50.0-valmin)/(valmax-valmin), 1.0, 1.0 ] )
blue.append( [ (-50.0-valmin)/(valmax-valmin), 0.0, 0.0 ] )
red.append( [ (-40.0-valmin)/(valmax-valmin), 0.0, 0.0 ] )
green.append( [ (-40.0-valmin)/(valmax-valmin), 0.0, 0.0 ] )
blue.append( [ (-40.0-valmin)/(valmax-valmin), 0.4, 0.4 ] )
red.append( [ (-30.0-valmin)/(valmax-valmin), 0.0, 0.8 ] )
green.append( [ (-30.0-valmin)/(valmax-valmin), 1.0, 0.8 ] )
blue.append( [ (-30.0-valmin)/(valmax-valmin), 1.0, 0.8 ] )
red.append( [ 1.0, 0.0, 0.0 ] )
green.append( [ 1.0, 0.0, 0.0 ] )
blue.append( [ 1.0, 0.0, 0.0 ] )
cdict = {'red': red, 'green': green, 'blue': blue}
ctbl = LinearSegmentedColormap('custom', cdict)

cdict = {'red': ((0.0, 0.0, 0.0),
                 (.001, 1.00, 1.00),
                 (.107, 1.00, 1.00),
                 (.113, 0.498, 0.498),
                 (.173, 1.00, 1.00),
                 (.179, 0.902, 0.902),
                 (.227, 0.102, 0.102),
                 (.233, 0.00, 0.00),
                 (.287, 0.902, 0.902),
                 (.293, 1.00, 1.00),
                 (.346, 1.00, 1.00),
                 (.352, 1.00, 1.00),
                 (.406, 0.101, 0.101),
                (.412, 0.00, 0.00),
                 (.481, 0.00, 0.00),
                 (.484, 0.00, 0.00),
                 (.543, 0.00, 0.00),
                 (.546, 0.773, 0.773),
                 (.994, 0.012, 0.012),
                 (.997, 0.004, 0.004),
                 (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 0.0),
                 (.001, 1.00, 1.00),
                 (.107, 1.00, 1.00),
                 (.113, 0.00, 0.00),
                 (.173, 0.498, 0.498),
                 (.179, 0.902, 0.902),
                 (.227, 0.102, 0.102),
                 (.233, 0.00, 0.00),
                 (.287, 0.00, 0.00),
                 (.293, 0.00, 0.00),
                 (.346, 0.902, 0.902),
                 (.352, 1.00, 1.00),
                 (.406, 1.00, 1.00),
                 (.412, 1.00, 1.00),
                 (.481, 0.00, 0.00),
                 (.484, 0.00, 0.00),
                 (.543, 1.00, 1.00),
                 (.546, 0.773, 0.773),
                 (.994, 0.012, 0.012),
                 (.997, 0.004, 0.004),
                   (1.0, 0.0, 0.0)),
         'blue': ((0.0, 0.00, 0.00),
                 (.001, 1.00, 1.00),
                 (.107, 0.00, 0.00),
                 (.113, 0.498, 0.498),
                 (.173, 0.786, 0.786),
                 (.179, 0.902, 0.902),
                 (.227, 0.102, 0.102),
                 (.233, 0.00, 0.00),
                 (.287, 0.00, 0.00),
                 (.293, 0.00, 0.00),
                 (.346, 0.00, 0.00),
                 (.352, 0.00, 0.00),
                 (.406, 0.00, 0.00),
                 (.412, 0.00, 0.00),
                 (.481, 0.451, 0.451),
                 (.484, 0.451, 0.451),
                 (.543, 1.00, 1.00),
                 (.546, 0.773, 0.773),
                 (.994, 0.012, 0.012),
                 (.997, 0.004, 0.004),
                  (1.0, 0.0, 0.0))}


IR_cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,2048)

In [ ]:
#WV_cmap = ctables.registry.get_colortable('WVCIMSS')

### Here I'd like to make a block of code to directly download files via Amazon AWS and rclone...

## Walk the directories to find the files and populate new lists for each channel for the current day

In [ ]:
JulianDate = now.strftime('%j') 
print JulianDate

GOES_sample_path = '/path/to/downloaded/files/' # Use full path to file   

# Assumed that these are radiance files for the CONUS extent; customize as necessary    
GOES_samples_2 = []
for name in glob.glob(GOES_sample_path+'OR*L1b*RadC*C02*s'+'{:%Y}'.format(now)+str(JulianDate)+'*.nc'):
    GOES_samples_2.append(name)
GOES_samples_2 = [os.path.join(GOES_sample_path, s) for s in GOES_samples_2]
GOES_samples_2 = sorted(GOES_samples_2, key=lambda x: int(re.sub('\D', '', x)))

GOES_samples_9 = []
for name in glob.glob(GOES_sample_path+'OR*L1b*RadC*C09*s'+'{:%Y}'.format(now)+str(JulianDate)+'*.nc'):
    GOES_samples_9.append(name)
GOES_samples_9 = [os.path.join(GOES_sample_path, s) for s in GOES_samples_9]
GOES_samples_9 = sorted(GOES_samples_9, key=lambda x: int(re.sub('\D', '', x)))

GOES_samples_13 = []
for name in glob.glob(GOES_sample_path+'OR*L1b*RadC*C13*s'+'{:%Y}'.format(now)+str(JulianDate)+'*.nc'):
    GOES_samples_13.append(name)
GOES_samples_13 = [os.path.join(GOES_sample_path, s) for s in GOES_samples_13]
GOES_samples_13 = sorted(GOES_samples_13, key=lambda x: int(re.sub('\D', '', x)))

# Find and convert Julian day to date 
import datetime as DT
dt = datetime(2018,1,1)
dtdelta = DT.timedelta(days=int(GOES_samples_13[0][59:62])-1)
day = dt + dtdelta

GOES_samples_13

<h1>Red Band Visible Channel 2</h1>

In [ ]:
VIS_path = GOES_samples_2[0]
print VIS_path
vis_name = VIS_path[-76:]
print vis_name
vis_title = vis_name[27:31]+"_"+vis_name[31:34]+"_"+vis_name[34:36]+"_"+vis_name[36:38]
print vis_title
nc_VIS = Dataset(VIS_path)
data_VIS = nc_VIS.variables['Rad'][:]

sat_h = nc_VIS.variables['goes_imager_projection'].perspective_point_height

# Satellite longitude
#sat_lon = nc.variables['goes_imager_projection'].longitude_of_projection_origin

# Satellite sweep
#sat_sweep = nc.variables['goes_imager_projection'].sweep_angle_axis

# The projection x and y coordinates equals
# the scanning angle (in radians) multiplied by the satellite height (http://proj4.org/projections/geos.html)
X = nc_VIS.variables['x'][:] * sat_h
Y = nc_VIS.variables['y'][:] * sat_h

proj_var = nc_VIS.variables['goes_imager_projection']
sat_height = proj_var.perspective_point_height
central_lon = proj_var.longitude_of_projection_origin
semi_major = proj_var.semi_major_axis
semi_minor = proj_var.semi_minor_axis

<h1>Mid-Level Water Vapor Channel 9</h1>

In [ ]:
WV_path = GOES_samples_9[0]
print WV_path
wv_name = WV_path[-76:]
print wv_name
wv_title = wv_name[27:31]+"_"+wv_name[31:34]+"_"+wv_name[34:36]+"_"+wv_name[36:38]
print wv_title
nc_WV = Dataset(WV_path)
data_WV = nc_WV.variables['Rad'][:]

<h1>Clean Infrared Channel 13</h1>

In [ ]:
IR_path = GOES_samples_13[0]
print IR_path
ir_name = IR_path[-76:]
print ir_name
ir_title = ir_name[27:31]+"_"+ir_name[31:34]+"_"+ir_name[34:36]+"_"+ir_name[36:38]
print ir_title
nc_IR = Dataset(IR_path)
data_IR = nc_IR.variables['Rad'][:]
fk1 = nc_IR.variables['planck_fk1'][0]
fk2 = nc_IR.variables['planck_fk2'][0]
bc1 = nc_IR.variables['planck_bc1'][0]
bc2 = nc_IR.variables['planck_bc2'][0]

data_IR = (fk2 / ( np.log((fk1 / data_IR) + 1 )) - bc1) / bc2

Begin map creation
------------------



In [ ]:
# Add state boundaries to plot
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Set Projection of Plot
globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)
#proj = ccrs.PlateCarree(central_longitude=central_lon,globe=globe) #,globe=globe

extent = [-130., -70, 20., 60.]

## We can bring up the University of Wisconsin's IR Image to verify ours

<img src="https://whirlwind.aos.wisc.edu/~wxp/goes16/ircm/conus/latest_conus_1.jpg">

<h2>---------------------------------------------------------------------------------------------------------</h2>

## We can define a map projection function that can take details of the different channels

In [ ]:
def Map(path,channel,band,title,date,data,savepath,my_cmap,X,Y,vmin=None,vmax=None):
    print path
    fig = plt.figure(figsize=(17., 11.))

# Add state boundaries to plot
    states_boundaries = cfeature.NaturalEarthFeature(category='cultural',
        name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

    country_borders = cfeature.NaturalEarthFeature(category='cultural',
        name='admin_0_countries',scale='50m', facecolor='none')

# Set Projection of Plot
    globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
    crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)
    plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Add the map and set the extent
    ax = plt.subplot(111, projection=plotcrs) 
        
# Find and convert Julian day to date    
    import datetime as DT
    dt = datetime(2018,1,1)
    dtdelta = DT.timedelta(days=int(path[59:62])-1)
    Day = dt + dtdelta

# Set the plot title    
    plt.title(title,loc='left',fontsize=16)
    plt.title(path[55:59]+' {0:%B %d}'.format(Day)+" "+path[62:64]+":"+path[64:66]+"Z", 
          fontsize=16,loc='right')

# Add state boundaries to plot
    ax.add_feature(states_boundaries, edgecolor='blue', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

    ax.set_extent([-130., -70, 20., 60.], ccrs.PlateCarree())
    
    img = ax.imshow(data[:,:],origin='upper',extent=(X.min(), X.max(), Y.min(), Y.max()),
        interpolation='nearest',transform=crs,cmap=my_cmap,vmin=vmin,vmax=vmax) 

    #plt.tight_layout()
    plt.show()
    
    outfile = savepath+ "GOES_Ch"+str(channel)+"_"+band+"_"+date+".png"

    fig.savefig(outfile,bbox_inches='tight',dpi=120)
    return Day

## We can now imput data from the different channels and have our generic map maker function run on each to produce our satellite images!

### Map Arguments: Map(path, channel, band, title, data, savepath, my_cmap, X, Y, vmin=None, vmax=None)

In [ ]:
# Map(path,channel,band,title,data,savepath,my_cmap,X,Y,vmin=None,vmax=None)

date = ir_title
title = 'GOES-16 Ch13 - Clean Infrared'
channel=13
band = "IR"
Map(GOES_samples_13[0],channel,band,title,date,data_IR,im_save_path,IR_cmap,X,Y,vmin=162.,vmax=330.)

#date = wv_title
channel = 9
band = 'WV'
title = 'GOES-16 Ch9 - Water Vapor'
Map(GOES_samples_9[0],channel,band,title,date,data_WV,im_save_path,'gist_gray_r',X,Y)

#date = vis_title
channel = 2
band = 'VIS'
title = 'GOES-16 Ch2 - Red'
#Map(GOES_samples_2[0],channel,band,title,date,data_VIS,im_save_path,'Greys_r',X,Y)

# We can also take the lists of files we created earlier and create animated gifs

## Ch 13

In [ ]:
#%%time
# Each plot takes roughly 13 seocnds, keep that in mind when trying to plot multiple hours...
print "Here we go..."
for i in range(len(GOES_samples_13)):
  
    IR_path = GOES_samples_13[i]
    date = IR_path[55:59]+"_"+IR_path[59:62]+"_"+IR_path[62:64]+"_"+IR_path[64:66]
    nc = Dataset(IR_path)
 
# Extract the Brightness Temperature values from the NetCDF
    data = nc.variables['Rad'][:]

# Recalculate the data so we can plot infrared...    
    fk1 = nc.variables['planck_fk1'][0]
    fk2 = nc.variables['planck_fk2'][0]
    bc1 = nc.variables['planck_bc1'][0]
    bc2 = nc.variables['planck_bc2'][0]
    data = (fk2 / ( np.log((fk1 / data) + 1 )) - bc1) / bc2

# Initiate the figure    
    fig = plt.figure(figsize=(17., 11.))

# Add the map
    ax = plt.subplot(111, projection=plotcrs)

# Set plot title    
    plt.title('GOES-16 Ch13 - Clean Infrared',loc='left',fontsize=16)
    plt.title(IR_path[55:59]+' {0:%B %d}'.format(day)+" "+IR_path[62:64]+":"+IR_path[64:66]+"Z", 
          fontsize=16,loc='right')

# Add state boundaries to plot
    ax.add_feature(states_provinces, edgecolor='blue', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Set the extent of Lat/Lon    
    ax.set_extent(extent, ccrs.PlateCarree())

# Plot the data    
    IR_img = ax.imshow(data[:,:],origin='upper',extent=(X.min(), X.max(), Y.min(), Y.max()),
    interpolation='nearest',transform=crs,cmap=IR_cmap,vmin=162.,vmax=330.) #cmap=WV_cmap  transform=plotcrs

# Set saved figure filename and save fig
    outfile = IRPATH+"IR_"+date+".png"   # change path ranges for your own file locations... 
    fig.savefig(outfile,bbox_inches='tight',dpi=120)
    ax.axis('off')
    plt.close(fig)
    
print "Images done"
print "gif activated..."

# Set a new list to hold all the images for this channel
files = []
# searching for the files with * 
for name in glob.glob(IRPATH+'*.png'):
    #print name
    files.append(name)

# Make sure our files are in chronological order so our animation is accurate
files = sorted(files, key=lambda x: int(re.sub('\D', '', x)))

# Set a new list full of the images so we can create an animation
images = []
for filename in files:
    images.append(imageio.imread(filename))
imageio.mimsave(IRPATH+"IR_"+IR_path[55:59]+"_"+IR_path[59:62]+".gif", 
                images,duration=0.3)
print "gif done."

## Ch 9

In [ ]:
print "Here we go..."
for i in range(len(GOES_samples_9)):
  
    IR_path = GOES_samples_9[i]
    date = IR_path[55:59]+"_"+IR_path[59:62]+"_"+IR_path[62:64]+"_"+IR_path[64:66]
    nc = Dataset(IR_path)
 
# Extract the Brightness Temperature values from the NetCDF
    data = nc.variables['Rad'][:]
    
# Initiate the figure    
    fig = plt.figure(figsize=(17., 11.))

# Add the map
    ax = plt.subplot(111, projection=plotcrs)

# Set plot title  
    plt.title('GOES-16 Ch9 - Water Vapor',loc='left',fontsize=16)
    plt.title(IR_path[55:59]+' {0:%B %d}'.format(day)+" "+IR_path[62:64]+":"+IR_path[64:66]+"Z", 
          fontsize=16,loc='right')

# Add state boundaries to plot
    ax.add_feature(states_provinces, edgecolor='blue', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Se the extent of Lat/Lon    
    ax.set_extent(extent, ccrs.PlateCarree())

# Plot the data    
    WV_img = ax.imshow(data[:,:],origin='upper',extent=(X.min(), X.max(), Y.min(), Y.max()),
    interpolation='nearest',transform=crs,cmap='gist_gray_r') #cmap=WV_cmap  transform=plotcrs

# Set saved figure filename and save fig
    outfile = WVPATH+"WV_"+date+".png"
    fig.savefig(outfile,bbox_inches='tight',dpi=120)
    ax.axis('off')
    plt.close(fig)

print "Images done"
print "gif activated..."

# Set a new list to hold all the images for this channel
files = []
# searching for the files with * 
for name in glob.glob(WVPATH+'*.png'):
    #print name
    files.append(name)

# Make sure our files are in chronological order so our animation is accurate
files = sorted(files, key=lambda x: int(re.sub('\D', '', x)))
    
# Set a new list full of the images so we can create an animation
images = []
for filename in files:
    images.append(imageio.imread(filename))
imageio.mimsave(WVPATH+"WV_"+IR_path[55:59]+"_"+IR_path[59:62]+".gif", 
                images,duration=0.3)
print "gif done."

## Ch 2 - Its currently commented out because it takes a looong time to run...